In [1]:
# ============================================
# Notebook: etl_covid_to_sql.ipynb
# Objetivo: Pipeline ETL - CSV → DataFrame → SQL Server
# Data: 16/10/2025
# Dataset: COVID-19 Brasil (dados públicos)
# ============================================

# CÉLULA 1: Configurar ambiente
import os
import sys

# Configurações do Spark
python_path = sys.executable
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path
os.environ['SPARK_LOCAL_HOSTNAME'] = 'localhost'

print("=== CONFIGURAÇÃO DO AMBIENTE ===")
print(f"✓ Python: {sys.version.split()[0]}")
print(f"✓ Executável: {python_path}")
print("="*60)

# Importar bibliotecas
print("\n=== IMPORTANDO BIBLIOTECAS ===")
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import urllib.request
print("✓ Bibliotecas importadas!")

# ============================================

# CÉLULA 2: Criar Spark Session
print("\n=== CRIANDO SPARK SESSION ===")

spark = SparkSession.builder \
    .appName("ETL_COVID_to_SQL") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

print(f"✓ Spark Session criada!")
print(f"✓ Versão: {spark.version}")
print(f"✓ App Name: {spark.sparkContext.appName}")

# ============================================

# CÉLULA 3: EXTRACT - Baixar dados públicos de COVID-19
print("\n=== FASE 1: EXTRACT (Extração) ===")

# URL do dataset público (COVID-19 Brasil - atualizado diariamente)
url = "https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv"

# Criar diretório para dados brutos
os.makedirs("../data/raw", exist_ok=True)
raw_file = "../data/raw/covid19_brazil_raw.csv"

print(f"📥 Baixando dados de: {url}")
try:
    urllib.request.urlretrieve(url, raw_file)
    print(f"✓ Dados baixados: {raw_file}")
    
    # Verificar tamanho do arquivo
    file_size = os.path.getsize(raw_file) / (1024 * 1024)  # MB
    print(f"✓ Tamanho: {file_size:.2f} MB")
except Exception as e:
    print(f"❌ Erro ao baixar: {e}")
    print("⚠️ Usando arquivo de exemplo local...")

# ============================================

# CÉLULA 4: Ler CSV com Spark
print("\n=== LENDO CSV COM SPARK ===")

# Ler CSV
df_raw = spark.read.csv(
    raw_file,
    header=True,
    inferSchema=True,
    sep=","
)

print(f"✓ CSV lido com sucesso!")
print(f"✓ Total de registros: {df_raw.count():,}")
print(f"✓ Total de colunas: {len(df_raw.columns)}")

# Mostrar primeiras linhas
print("\n📊 Primeiras linhas dos dados brutos:")
df_raw.show(5, truncate=False)

# ============================================

# CÉLULA 5: Explorar dados brutos
print("\n=== ANÁLISE EXPLORATÓRIA ===")

# Schema
print("\n1. Schema dos dados:")
df_raw.printSchema()

# Colunas
print("\n2. Colunas disponíveis:")
for i, col in enumerate(df_raw.columns, 1):
    print(f"   {i}. {col}")

# Estatísticas básicas
print("\n3. Estatísticas descritivas:")
df_raw.describe().show()

# Valores nulos
print("\n4. Contagem de valores nulos por coluna:")
from pyspark.sql.functions import col, sum as spark_sum, count, when

null_counts = df_raw.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df_raw.columns
])
null_counts.show()

# ============================================

# CÉLULA 6: TRANSFORM - Limpeza e Transformação
print("\n=== FASE 2: TRANSFORM (Transformação) ===")

print("🧹 Iniciando limpeza dos dados...")

# 1. Selecionar apenas colunas relevantes
colunas_interesse = [
    'date',           # Data
    'state',          # Estado
    'city',           # Cidade
    'totalCases',     # Total de casos
    'newCases',       # Novos casos
    'deaths',         # Óbitos totais
    'newDeaths'       # Novos óbitos
]

# Verificar se as colunas existem
colunas_disponiveis = [c for c in colunas_interesse if c in df_raw.columns]
print(f"✓ Colunas selecionadas: {colunas_disponiveis}")

df_clean = df_raw.select(colunas_disponiveis)

# 2. Remover linhas com valores nulos nas colunas críticas
print("\n🔍 Removendo valores nulos...")
registros_antes = df_clean.count()
df_clean = df_clean.dropna(subset=['date', 'state', 'city'])
registros_depois = df_clean.count()
print(f"✓ Registros removidos: {registros_antes - registros_depois:,}")
print(f"✓ Registros restantes: {registros_depois:,}")

# 3. Remover duplicatas
print("\n🔍 Removendo duplicatas...")
registros_antes = df_clean.count()
df_clean = df_clean.dropDuplicates(['date', 'state', 'city'])
registros_depois = df_clean.count()
print(f"✓ Duplicatas removidas: {registros_antes - registros_depois:,}")

# 4. Converter tipos de dados
print("\n🔄 Convertendo tipos de dados...")
df_clean = df_clean.withColumn('date', to_date(col('date'), 'yyyy-MM-dd'))
df_clean = df_clean.withColumn('totalCases', col('totalCases').cast('integer'))
df_clean = df_clean.withColumn('newCases', col('newCases').cast('integer'))
df_clean = df_clean.withColumn('deaths', col('deaths').cast('integer'))
df_clean = df_clean.withColumn('newDeaths', col('newDeaths').cast('integer'))

# 5. Adicionar colunas calculadas
print("\n➕ Adicionando colunas calculadas...")
df_clean = df_clean.withColumn('year', year(col('date')))
df_clean = df_clean.withColumn('month', month(col('date')))
df_clean = df_clean.withColumn('data_carga', current_timestamp())

# 6. Renomear colunas para português
print("\n🔤 Renomeando colunas...")
df_clean = df_clean \
    .withColumnRenamed('date', 'data') \
    .withColumnRenamed('state', 'estado') \
    .withColumnRenamed('city', 'cidade') \
    .withColumnRenamed('totalCases', 'casos_totais') \
    .withColumnRenamed('newCases', 'casos_novos') \
    .withColumnRenamed('deaths', 'obitos_totais') \
    .withColumnRenamed('newDeaths', 'obitos_novos') \
    .withColumnRenamed('year', 'ano') \
    .withColumnRenamed('month', 'mes')

print("✓ Transformação concluída!")

# ============================================

# CÉLULA 7: Visualizar dados limpos
print("\n=== DADOS LIMPOS ===")

print("\n📊 Schema final:")
df_clean.printSchema()

print("\n📊 Amostra dos dados limpos:")
df_clean.show(10)

print("\n📊 Estatísticas dos dados limpos:")
df_clean.select('casos_totais', 'casos_novos', 'obitos_totais', 'obitos_novos').describe().show()

# ============================================

# CÉLULA 8: Análises básicas
print("\n=== ANÁLISES BÁSICAS ===")

# 1. Total por estado
print("\n1. Top 10 Estados com mais casos:")
df_clean.groupBy('estado') \
    .agg(
        sum('casos_totais').alias('total_casos'),
        sum('obitos_totais').alias('total_obitos')
    ) \
    .orderBy(col('total_casos').desc()) \
    .limit(10) \
    .show()

# 2. Evolução temporal
print("\n2. Casos por ano:")
df_clean.groupBy('ano') \
    .agg(
        sum('casos_novos').alias('casos_ano'),
        sum('obitos_novos').alias('obitos_ano')
    ) \
    .orderBy('ano') \
    .show()

# 3. Dados mais recentes
print("\n3. Dados mais recentes (últimas 5 datas):")
df_clean.orderBy(col('data').desc()).limit(5).show()

# ============================================

# CÉLULA 9: Salvar dados limpos
print("\n=== SALVANDO DADOS LIMPOS ===")

# Criar diretório para dados processados
os.makedirs("../data/processed", exist_ok=True)

# Salvar como CSV (usando Pandas para evitar problema do Hadoop)
print("💾 Salvando como CSV...")
processed_file = "../data/processed/covid19_brazil_clean.csv"
df_clean_pandas = df_clean.toPandas()
df_clean_pandas.to_csv(processed_file, index=False)
print(f"✓ CSV salvo: {processed_file}")
print(f"✓ Registros salvos: {len(df_clean_pandas):,}")

# Salvar como Parquet (formato mais eficiente)
print("\n💾 Salvando como Parquet...")
parquet_path = "../data/processed/covid19_brazil_clean.parquet"
df_clean_pandas.to_parquet(parquet_path, index=False)
print(f"✓ Parquet salvo: {parquet_path}")

# ============================================

# CÉLULA 10: LOAD - Preparar para SQL Server
print("\n=== FASE 3: LOAD (Carregamento) ===")

print("📋 Preparando dados para SQL Server...")

# Para o SQL Server, vamos criar uma amostra menor (últimos 1000 registros)
print("\n🔽 Criando amostra para SQL Server...")
df_sample = df_clean.orderBy(col('data').desc()).limit(1000)

print(f"✓ Amostra criada: {df_sample.count():,} registros")
print("\n📊 Primeiras linhas da amostra:")
df_sample.show(5)

# ============================================

# CÉLULA 11: Criar script SQL para tabela
print("\n=== GERANDO SCRIPT SQL ===")

sql_script = """
-- ============================================
-- Script: create_table_covid19.sql
-- Gerado automaticamente pelo pipeline ETL
-- Data: 16/10/2025
-- ============================================

USE datalake_local;
GO

-- Limpar tabela se existir
IF OBJECT_ID('dbo.covid19_brazil', 'U') IS NOT NULL
    DROP TABLE dbo.covid19_brazil;
GO

-- Criar tabela
CREATE TABLE dbo.covid19_brazil (
    id INT PRIMARY KEY IDENTITY(1,1),
    data DATE NOT NULL,
    estado VARCHAR(2),
    cidade VARCHAR(200),
    casos_totais INT,
    casos_novos INT,
    obitos_totais INT,
    obitos_novos INT,
    ano INT,
    mes INT,
    data_carga DATETIME DEFAULT GETDATE()
);
GO

-- Criar índices para melhor performance
CREATE INDEX idx_data ON dbo.covid19_brazil(data);
CREATE INDEX idx_estado ON dbo.covid19_brazil(estado);
CREATE INDEX idx_cidade ON dbo.covid19_brazil(cidade);
GO

PRINT 'Tabela covid19_brazil criada com sucesso!';
GO
"""

# Salvar script SQL
sql_file = "../sql/create_table_covid19.sql"
os.makedirs("../sql", exist_ok=True)
with open(sql_file, 'w', encoding='utf-8') as f:
    f.write(sql_script)

print(f"✓ Script SQL salvo: {sql_file}")
print("\n📝 Conteúdo do script:")
print(sql_script)

# ============================================

# CÉLULA 12: Gerar INSERT SQL
print("\n=== GERANDO DADOS PARA INSERÇÃO ===")

# Converter amostra para formato de INSERT
print("📝 Gerando comandos INSERT...")

# Pegar apenas alguns registros para exemplo
df_insert = df_sample.limit(10).toPandas()

insert_statements = []
insert_statements.append("-- Inserir dados de exemplo\n")
insert_statements.append("USE datalake_local;\nGO\n\n")

for _, row in df_insert.iterrows():
    insert = f"""INSERT INTO dbo.covid19_brazil (data, estado, cidade, casos_totais, casos_novos, obitos_totais, obitos_novos, ano, mes) VALUES ('{row['data']}', '{row['estado']}', '{row['cidade']}', {row['casos_totais']}, {row['casos_novos']}, {row['obitos_totais']}, {row['obitos_novos']}, {row['ano']}, {row['mes']});"""
    insert_statements.append(insert + "\n")

insert_statements.append("\nGO\n")
insert_statements.append("\nPRINT 'Dados inseridos com sucesso!';\nGO")

# Salvar inserts
insert_file = "../sql/insert_covid19_sample.sql"
with open(insert_file, 'w', encoding='utf-8') as f:
    f.writelines(insert_statements)

print(f"✓ Script de INSERT salvo: {insert_file}")
print(f"✓ Total de INSERTs: {len(df_insert)}")

# Mostrar exemplo
print("\n📝 Exemplo de INSERT gerado:")
print(''.join(insert_statements[:5]))

# ============================================

# CÉLULA 13: Resumo do Pipeline ETL
print("\n" + "="*60)
print("=== RESUMO DO PIPELINE ETL ===")
print("="*60)

print("\n✅ EXTRACT (Extração):")
print(f"   • Fonte: COVID-19 Brasil (GitHub)")
print(f"   • Arquivo baixado: {raw_file}")
print(f"   • Registros originais: {df_raw.count():,}")

print("\n✅ TRANSFORM (Transformação):")
print(f"   • Colunas selecionadas: {len(df_clean.columns)}")
print(f"   • Registros após limpeza: {df_clean.count():,}")
print(f"   • Transformações aplicadas:")
print(f"     - Remoção de nulos")
print(f"     - Remoção de duplicatas")
print(f"     - Conversão de tipos")
print(f"     - Colunas calculadas (ano, mês)")
print(f"     - Renomeação para português")

print("\n✅ LOAD (Carregamento):")
print(f"   • CSV processado: {processed_file}")
print(f"   • Parquet gerado: {parquet_path}")
print(f"   • Script SQL: {sql_file}")
print(f"   • Inserts SQL: {insert_file}")
print(f"   • Amostra para SQL: {df_sample.count():,} registros")

print("\n" + "="*60)
print("🎉 PIPELINE ETL COMPLETO!")
print("="*60)

print("\n📋 PRÓXIMOS PASSOS:")
print("1. Execute o script SQL no SSMS para criar a tabela")
print("2. Execute o script de INSERT para popular com dados de exemplo")
print("3. Ou use uma ferramenta de ETL para carregar todos os dados")

# ============================================

# CÉLULA 14: Parar Spark Session
print("\n=== FINALIZANDO ===")
# spark.stop()  # Descomente para encerrar
print("✓ Pipeline concluído com sucesso!")
print("✓ Spark Session mantida ativa para análises adicionais")

=== CONFIGURAÇÃO DO AMBIENTE ===
✓ Python: 3.10.0
✓ Executável: C:\Users\Gabriel\Documents\ml-data-studying-projects\venv_spark\Scripts\python.exe

=== IMPORTANDO BIBLIOTECAS ===
✓ Bibliotecas importadas!

=== CRIANDO SPARK SESSION ===
✓ Spark Session criada!
✓ Versão: 4.0.1
✓ App Name: ETL_COVID_to_SQL

=== FASE 1: EXTRACT (Extração) ===
📥 Baixando dados de: https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv
✓ Dados baixados: ../data/raw/covid19_brazil_raw.csv
✓ Tamanho: 0.54 MB

=== LENDO CSV COM SPARK ===
✓ CSV lido com sucesso!
✓ Total de registros: 5,596
✓ Total de colunas: 14

📊 Primeiras linhas dos dados brutos:
+-------+-----+----------------------+-------+------+----------+---------------------------+-------------------------------+--------------------+-------+----------+--------+---------+--------------+
|country|state|city                  |ibgeID |deaths|totalCases|deaths_per_100k_inhabitants|totalCases_per_100k_inhabitants|deaths_by_totalCases